In [2]:
from nltk.tokenize import TreebankWordTokenizer
from pymystem3 import Mystem
from nltk.tokenize import sent_tokenize

fp = open('ok_go.txt')
data = fp.read()
sent_split = sent_tokenize(data)
len(sent_split)

5324

In [3]:
tokenizer = TreebankWordTokenizer()

In [4]:
doc_tokens = []
for line in sent_split:    
    doc_tokens += [sorted(tokenizer.tokenize(line.lower()))]

In [5]:
from collections import Counter
import nltk
puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';'))
stopwords = nltk.corpus.stopwords.words('russian')
all_doc_tokens = sum(doc_tokens, [])

word_list = [x for x in all_doc_tokens 
             if x not in puncs 
             if x not in stopwords]

lexicon = sorted(set(word_list))

print('Объем лексикона (словоформы) без стопслов:', len(lexicon))
print('Количество токенов без стопслов и знаков препинания:', len(word_list))

Объем лексикона (словоформы) без стопслов: 16073
Количество токенов без стопслов и знаков препинания: 42788


In [92]:
def clean_chapters(pass):
    pass
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                 ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9'))
    memo_lemmas_copy = []
    for sentence in sent_split:
        memo_lemmas_copy += [copy.copy(sentence)]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]
    df = pd.DataFrame(memo_lemmas_copy)
    return df

corpus = clean_chapters(data)
corpus

SyntaxError: invalid syntax (<ipython-input-92-167d8075305f>, line 1)

In [10]:
import copy
import pandas as pd

fp = open('Kundera_sentences.txt')
data = fp.read()
sent_split = sent_tokenize(data)
len(sent_split)

"""Splits text on sentences and saves them into a list without keeping chapters splitted. 
    Removes punctuation"""
def clean_sentences(text):
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                 ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'))
    stopwords = nltk.corpus.stopwords.words('russian')
    memo_lemmas_copy = []
    for sentence in sent_split:
        memo_lemmas_copy += [copy.copy(sentence)]
    memo_lemmas_copy = [(letter.replace('\n', ' ') for letter in sentence) #turn it off to split chapters
                       for sentence in memo_lemmas_copy]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]

    df = pd.DataFrame(memo_lemmas_copy, columns=['text'])
    return df

corpus = clean_sentences(data)
corpus

AttributeError: 'generator' object has no attribute 'split'

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=corpus.text).toarray()
len(tfidf.vocabulary_)

tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape

(5192, 7680)

In [116]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
index = ['sentence{}'.format(i) for i in range(len(corpus))]
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(106)

,topic0,topic1,topic2,topic3,...,topic12,topic13,topic14,topic15
sentence0,-0.087,-0.046,0.015,0.022,...,0.000,-0.023,-0.010,-0.018
sentence1,0.002,-0.026,0.030,-0.026,...,-0.035,-0.019,0.026,0.021
sentence2,-0.067,0.043,0.006,-0.019,...,-0.019,0.001,0.116,-0.049
sentence3,0.079,0.039,0.098,-0.127,...,0.038,0.012,-0.026,0.071
sentence4,0.045,-0.005,0.063,-0.077,...,0.003,0.057,0.079,-0.005
sentence5,-0.011,-0.085,0.045,0.001,...,-0.013,0.037,0.042,-0.016
sentence6,-0.047,-0.097,-0.004,0.023,...,-0.003,0.016,0.025,-0.004
sentence7,-0.081,-0.018,0.020,-0.040,...,0.052,0.050,-0.003,-0.005
sentence8,0.055,0.056,0.036,-0.100,...,-0.010,-0.004,-0.060,0.052
sentence9,-0.058,-0.068,-0.003,-0.039,...,0.001,0.009,-0.018,0.026


In [117]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
terms

('a',
 'alter',
 'anders',
 'animata',
 'animatae',
 'auch',
 'avoir',
 'beutel',
 'co',
 'com',
 'compassion',
 'conkreto',
 'corpus',
 'curriculum',
 'das',
 'de',
 'delicti',
 'dem',
 'der',
 'ego',
 'einmal',
 'entschluss',
 'entschlusse',
 'es',
 'femme',
 'gefasste',
 'gewicht',
 'heraus',
 'in',
 'ist',
 'ja',
 'keinmal',
 'konnte',
 'la',
 'land',
 'machina',
 'machinae',
 "man's",
 'medkansla',
 'mit',
 'mitgefuhl',
 'muss',
 'negationem',
 'no',
 'passio',
 'per',
 'pieta',
 'pitie',
 'pity',
 'pour',
 'priori',
 'schwer',
 'schwerste',
 'sein',
 'soucit',
 'une',
 'vitae',
 'wspolczucie',
 'а',
 'аа',
 'абажур',
 'аборт',
 'абсолютно',
 'абсолютный',
 'абстрактный',
 'абсурдный',
 'авантюра',
 'авария',
 'август',
 'автобус',
 'автомат',
 'автоматически',
 'автомобиль',
 'автор',
 'авторитарный',
 'авторучка',
 'автостоянка',
 'агент',
 'агрессивность',
 'агрессивный',
 'агрессия',
 'ад',
 'адам',
 'адрес',
 'адресат',
 'адресовать',
 'азия',
 'аист',
 'академия',
 'аккорд',

In [118]:
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns = 8
weights.head(10).round(3)

,a,alter,anders,animata,...,ясность,ясный,ящик,…
topic0,-0.000,0.000,-0.000,0.000,...,-0.001,-0.002,0.004,-0.000
topic1,-0.000,-0.001,0.000,-0.000,...,-0.000,-0.002,0.000,0.006
topic2,0.000,0.000,-0.000,-0.001,...,0.000,0.001,-0.001,-0.001
topic3,-0.001,0.000,-0.001,-0.001,...,0.000,-0.001,-0.002,0.000
topic4,-0.000,0.001,0.000,-0.001,...,0.001,-0.001,0.001,-0.009
topic5,0.000,0.001,-0.000,-0.001,...,0.000,0.000,-0.001,-0.001
topic6,0.000,-0.001,0.001,0.000,...,-0.001,0.001,-0.001,-0.001
topic7,0.001,-0.001,-0.000,0.002,...,-0.000,-0.001,0.002,-0.006
topic8,-0.000,-0.002,-0.000,0.000,...,-0.001,-0.004,0.000,0.003
topic9,-0.000,-0.001,-0.001,0.000,...,0.000,-0.002,-0.003,0.002


In [119]:
pd.options.display.max_columns = 12
deals = weights['двойственность душа тело легкость женщина душа тяжесть'.split()].round(3)*100
deals

,двойственность,душа,тело,легкость,женщина,душа,тяжесть
topic0,-0.1,1.0,2.2,-0.5,1.8,1.0,-0.8
topic1,-0.1,-1.5,-2.3,-0.2,-0.9,-1.5,-0.7
topic2,0.1,1.3,3.0,0.4,-2.9,1.3,0.1
topic3,-0.0,-1.7,-2.3,-0.3,-1.6,-1.7,-0.3
topic4,0.0,-1.9,-2.5,0.2,-2.0,-1.9,0.1
topic5,-0.0,-1.2,-2.0,0.6,-1.8,-1.2,-0.3
topic6,-0.1,0.0,-0.6,-0.4,0.0,0.0,-0.3
topic7,-0.1,0.7,-1.0,0.0,1.6,0.7,0.4
topic8,-0.0,0.7,1.8,-1.0,3.6,0.7,-0.4
topic9,-0.3,-2.2,-4.6,-0.2,-1.7,-2.2,-0.4


In [120]:
deals.T.sum()

topic0     4.600000e+00
topic1    -7.200000e+00
topic2     3.300000e+00
topic3    -7.900000e+00
topic4    -8.000000e+00
topic5    -5.900000e+00
topic6    -1.400000e+00
topic7     2.300000e+00
topic8     5.400000e+00
topic9    -1.160000e+01
topic10   -5.100000e+00
topic11    3.400000e+00
topic12    1.000000e+01
topic13    2.900000e+00
topic14   -1.100000e+00
topic15    1.110223e-16
dtype: float64

In [128]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
sentence0,-0.087,-0.046,0.015,0.022,0.042,0.022,...,-0.023,-0.033,0.009,-0.024,-0.011,-0.015
sentence1,0.002,-0.026,0.030,-0.026,-0.094,0.003,...,-0.045,0.012,-0.032,-0.017,0.017,-0.003
sentence2,-0.067,0.043,0.007,-0.019,-0.043,0.020,...,-0.116,-0.041,0.013,0.003,0.089,-0.078
sentence3,0.079,0.039,0.098,-0.127,-0.012,-0.042,...,-0.034,0.024,0.039,0.013,-0.029,0.067
sentence4,0.045,-0.005,0.063,-0.078,-0.074,-0.059,...,0.001,0.051,-0.018,0.050,0.084,0.003
sentence5,-0.011,-0.085,0.045,0.000,-0.050,0.016,...,0.030,-0.005,-0.022,0.033,0.042,-0.008


In [124]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=corpus.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms

In [132]:
corpus.loc[50].text

'позвать он она сейчас она приезжать и предлагать он весь свой жизнь'

In [135]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
ldia.components_.shape

(16, 7680)

In [136]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
components.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
a,0.06,0.06,0.06,0.06,0.06,1.06,...,0.06,0.06,0.06,0.06,0.06,0.06
alter,0.06,0.06,0.06,0.06,0.06,0.06,...,0.06,0.06,2.06,0.06,0.06,0.06
anders,0.06,0.06,0.06,0.06,0.06,0.06,...,0.06,0.06,0.06,0.06,0.06,0.06


In [148]:
components.topic10.sort_values(ascending=False)[:10]

и       122.014786
он       61.834957
они      49.774384
с        47.112621
быть     45.787174
на       45.089923
что      44.518971
этот     40.264034
в        33.757264
она      31.402616
Name: topic10, dtype: float64